# Library Importation 

In [33]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from datetime import datetime
import pytz


# Fetching Options Data on yfinance

In [34]:
# Assuming you want to fetch options for Apple (AAPL)
ticker = yf.Ticker('AAPL')

# Get all available expiration dates
expirations = ticker.options

# Let's choose the first expiration date as an example
expiration = expirations[0]

# Download the options data for this expiration
options = ticker.option_chain(expiration)

# 'options' is a named tuple with 'calls' and 'puts'
call_options = options.calls

#call_options

# Displaying the first few call options
#print(call_options[['strike', 'lastPrice', 'lastTradeDate', 'impliedVolatility']].head())

df = pd.DataFrame(call_options[['strike', 'lastPrice', 'lastTradeDate', 'impliedVolatility']].head(10))
df

,strike,lastPrice,lastTradeDate,impliedVolatility
0,100.0,134.60,2025-01-14 18:59:54+00:00,0.00001
1,115.0,107.09,2025-01-22 17:02:48+00:00,0.00001
2,120.0,134.55,2024-12-27 16:24:46+00:00,0.00001
3,125.0,110.14,2025-01-14 18:59:54+00:00,0.00001
4,130.0,124.00,2024-12-27 16:52:13+00:00,0.00001
5,135.0,88.50,2025-01-22 20:24:15+00:00,0.00001
6,140.0,82.15,2025-01-22 18:30:15+00:00,0.00001
7,145.0,83.94,2025-01-16 20:43:16+00:00,0.00001
8,150.0,78.91,2025-01-17 19:19:35+00:00,0.00001
9,155.0,67.55,2025-01-22 17:22:29+00:00,0.00001


In [35]:
# Convert 'lastTradeDate' to datetime objects, ensuring they're in UTC
if df['lastTradeDate'].dt.tz is None:
    df['lastTradeDate'] = pd.to_datetime(df['lastTradeDate']).dt.tz_localize('UTC')
else:
    df['lastTradeDate'] = df['lastTradeDate'].dt.tz_convert('UTC')  # Convert if already tz-aware

# Current date for calculation in UTC
current_date = datetime.now(pytz.utc)

# Calculate days to expiration
df['daysToExpiration'] = (df['lastTradeDate'] - current_date).dt.days

# Convert days to years for Black-Scholes (assuming 365 days in a year)
df['timeToExpiration'] = df['daysToExpiration'] / 365.0

# Drop the original 'lastTradeDate' if no longer needed
df = df.drop('lastTradeDate', axis=1)

# Display cleaned DataFrame
print(df)

   strike  lastPrice  impliedVolatility  daysToExpiration  timeToExpiration
0   100.0     134.60            0.00001                -9         -0.024658
1   115.0     107.09            0.00001                -1         -0.002740
2   120.0     134.55            0.00001               -27         -0.073973
3   125.0     110.14            0.00001                -9         -0.024658
4   130.0     124.00            0.00001               -27         -0.073973
5   135.0      88.50            0.00001                -1         -0.002740
6   140.0      82.15            0.00001                -1         -0.002740
7   145.0      83.94            0.00001                -7         -0.019178
8   150.0      78.91            0.00001                -6         -0.016438
9   155.0      67.55            0.00001                -1         -0.002740


# Black-Scholes Model

In [36]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma* np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Newton-Raphson Method

In [37]:
def implied_volatility_newton(S, K, T, r, C, tol=1e-5, max_iter=100):
    sigma = 0.2 # Inital guess for volatility
    for _ in range(max_iter):
        price = black_scholes_call(S, K, T, r, sigma)
        vega_val = vega(S, K, T, r, sigma)
        diff = C - price
        if abs(diff) < tol:
            return sigma
        sigma = sigma + diff / vega_val
    return sigma
                          
    

# Bisection Method

In [38]:
def implied_volatility_bisection(S, K, T, r, C, tol=1e-5):
    low, high = 0.001, 1.0 # Assuming volatility within this range
    while high - low > tol:
        mid = (low + high) / 2
        price = black_scholes_call(S, K, T, r, mid)
        if price > C:
            high = mid
        else:
            low = mid
        return (low + high) / 2
    
S = 100    #Current stock price
K = 100    #Strike price
T = 1      # Time to expiration in years
r = 0.05   # Risk-Free rate
C = 20     # Market price of the call option

print("Implied Volatility (Newton-Raphson):", implied_volatility_newton(S, K, T, r, C))
print("Implied Volatility (Bisection):", implied_volatility_bisection(S, K, T, r, C))

Implied Volatility (Newton-Raphson): 0.4523401602501279
Implied Volatility (Bisection): 0.25075
